In [1]:
import os
import copy

import json
import networkx as nx
import numpy as np
import pickle

In [2]:
obj = 'crash'
n = 800

with open(f'./qqqhe/test_instances/1589214682686/Instance{obj}/VB=100/n={n}/{obj}_VB=100_n={n}_0.json') as f:
    dict = json.load(f)

G = nx.path_graph(n)
G.add_nodes_from(range(n, 2*n - 1))

for i in range(n, 2*n-1):
    G.add_edge(i, 2*n-2-i)

for i in G.nodes():
    neighbors = list(G.neighbors(i))
    G.nodes[i]['children'] = [x for x in neighbors if x > i]
    G.nodes[i]['parent'] = [x for x in neighbors if x < i]

# Leaves
for i in range(n):
    G.nodes[i+n-1]['l'] = dict['lbVar'][i]
    G.nodes[i+n-1]['u'] = dict['capacity'][i]
    G.nodes[i+n-1]['obj'] = obj
    G.nodes[i+n-1]['param'] = {'a': dict['cost_param_a'][i], 'b': dict['cost_param_b'][i]}

# Inner nodes
for i in range(n-1):
    G.nodes[i]['l'] = dict['lbNested'][n-i-1]
    G.nodes[i]['u'] = dict['ubNested'][n-i-1]
    G.nodes[i]['obj'] = 'zero'
    G.nodes[i]['param'] = {}

# Modify the bounds on `x(0)` to reflect bounds for the singleton `{x(0)}`
G.nodes[n-1]['l'] = max(G.nodes[n-1]['l'], dict['lbNested'][0])
G.nodes[n-1]['u'] = min(G.nodes[n-1]['u'], dict['ubNested'][0])

In [ ]:
from utils import greedy, integer_initial_vector

yinit = integer_initial_vector(n, R)
greedy(yinit, G)

In [3]:
R = G.nodes[0]['l']

# add random weights
for alpha in [0.01, 0.1, 1]:

    new_directory_path = f'data/{obj}-n{n}/o{alpha}'
    if not os.path.exists(new_directory_path):
        os.makedirs(new_directory_path)

    for t in range(1000):
        Gt = copy.deepcopy(G)
        for i in G.nodes():
            if 'a' in Gt.nodes[i]['param']:
                Gt.nodes[i]['param']['a'] += np.random.randn(1)[0] * alpha
                Gt.nodes[i]['param']['a'] = max(0, Gt.nodes[i]['param']['a']) # `a` must be non-negative
            if 'b' in Gt.nodes[i]['param']:
                Gt.nodes[i]['param']['b'] += np.random.randn(1)[0] * alpha    # `b` can be negative
        with open(f'{new_directory_path}/{t}.pkl', 'wb') as f:
            pickle.dump(Gt, f)

    #save params
    np.savez(f'{new_directory_path}/params.npz', n=n, R=R)